### Step-1 : Import Modules

In [1]:
import os, cv2, glob
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
import gradio as gr
from keras.models import load_model

c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step-2 : Read Images 

In [ ]:
# Initialize lists and label dictionary
images, labels = [], []
label_map = {"60": 0, "100": 1}

def load_images_from_folder(base_path, img_size=(80, 80)):   # Uniform Input Size
    for folder in glob.glob(os.path.join(base_path, '*')):
        print(f"{folder} 圖片讀取中…")
        label = os.path.basename(folder)
        # print(label)
        
        # Skip folders that aren't in label_map
        if label not in label_map:
            print(f"未定義的標籤: {label}，跳過")
            continue
        
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder, file)
                try:
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, img_size) 
                        images.append(img)   # RGB data for each images
                        labels.append(label_map[label])   # 1 for 100 and 0 for 60
                    else:
                        print(f"無法讀取圖片: {img_path}")
                except Exception as e:
                    print(f"讀取檔案時出錯: {img_path}, 錯誤: {e}")
            else:
                print(f"跳過非圖片檔案: {file}")

# Define the base path and load images
base_path = "C:/Photos"  # 修改此路徑
load_images_from_folder(base_path)

print(images)
print(labels)

print(f'圖片數量：{len(images)}')
print(f'標籤數量：{len(labels)}')


### Step-3 : Splitting Data into Training and Testing Sets

#### Step -3.1 : 20% for testing and 80% for training

In [3]:
train_feature,test_feature,train_label,test_label = \
train_test_split(images,labels,test_size=0.2)

#### Step -3.2 : 20% for testing and 80% for training features
##### Conver lists into NumPy arrays

In [4]:
train_feature=np.array(train_feature)
test_feature=np.array(test_feature)

#### Step -3.3 : 20% for testing and 80% for training labels

In [5]:
train_label=np.array(train_label)
test_label=np.array(test_label)

#### Step-3.4 : Normalizing Image Data, Dividing by 255 scales the values to a range of [0, 1].

In [6]:
train_feature = train_feature/255
test_feature = test_feature/255

#### Step-3.5 : Converting Labels to Categorical Format (One-Hot Encoding)

In [7]:
train_label = np_utils.to_categorical(train_label)
test_label = np_utils.to_categorical(test_label)

In [ ]:
model = Sequential()

model.add(Conv2D(filters=8, kernel_size=(5,5), padding='same',input_shape=(80, 80, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=(5,5),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5,5),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(x=train_feature, y=train_label, validation_split=0.2,epochs=20, batch_size=200, verbose=2)

scores = model.evaluate(test_feature, test_label)

print('\n準確率=',scores[1])
model.save('fan_model.h5')

In [9]:
def show_images_labels_predictions(images, labels,predictions,start_id, num=10):
    plt.figure(figsize=(12, 14))
    if num>25: num=25
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[start_id])
        if( len(predictions) > 0 ) :
            title = 'ai = ' + str(predictions[start_id])
            title += (' (o)' if predictions[start_id]== \
            labels[start_id] else ' (x)')
            title += '\nlabel = ' + str(labels[start_id])
        else :
            title = 'label = ' + str(labels[start_id])
        ax.set_title(title,fontsize=12)
        ax.set_xticks([]);ax.set_yticks([])
        start_id+=1
    plt.show()

In [2]:
files = glob.glob("C:/Photos/*" )  #修改照片路徑
print(files)
test_feature=[]
test_label=[]
dict_labels = {"60":0, "100":1}

for file in files:
    img=cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, dsize=(80,80))
    test_feature.append(img)
    label=file[10:13]
    test_label.append(dict_labels[label])
    test_feature = np.array(test_feature).reshape(len(test_feature),80,80,3).astype('float32')
    test_label = np.array(test_label)

    test_feature_n = test_feature

try:

    model = load_model('/content/fan_model.h5')
    prediction = model.predict(test_feature_n)
    prediction = np.argmax(prediction,axis=1)
    show_images_labels_predictions(test_feature,test_label,prediction,0,len(test_feature))
except:
    print("模型未建立!")



model = load_model("C:/Photos/fan_model.h5")  #修改fan_model.h5路徑


['C:/Photos\\100', 'C:/Photos\\60']


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
def fan60100(image):
    image = np.array(image.resize((80, 80))).astype("float32") / 255.0
    image = image.reshape(1, 80, 80, 3)
    prediction = model.predict(image).tolist()[0]
    class_names = ["60", "100"]
    return {class_names[i]: prediction[i] for i in range(2)}

inp = gr.Image(type="pil")
out = gr.Label(num_top_classes=2, label='預測結果')
grobj = gr.Interface(fn=fan60100, inputs=inp,outputs=out, title="圖片辨識")

grobj.launch(share=True)


# 模組跑完後終端機會提供一組本地應用的地址
# 例:* Running on local URL:  http://127.0.0.1:7860
# 將地址貼至瀏覽器即可打開

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://dbfe0acd6e5e08a63d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
